In [1]:
# import libraries

import pandas as pd
import sqlite3

### The Database can be found [here](https://drive.google.com/file/d/1_RRxnL3Xf-J6CBLEbJVL4mSR9GA0T0QI/view?usp=drive_link)

> Run the command below to see all the tables present in the SQLite DB.

```sql
SELECT name FROM sqlite_master;
```

<hr>
<br>

In [2]:
# connect to database
con = sqlite3.connect("week_2.db")
pd.read_sql_query("SELECT name FROM sqlite_master", con)

,name
0,customers
1,sqlite_sequence
2,orders
3,employee
4,department
5,students
6,sqlite_autoindex_students_1
7,products


## Qs 1: 

a. Given the table, `customers` and `orders` shown below, use a **subquery** to find the **customers** that did ***NOT*** place any orders. Display only the customer name(s).

**`customers`**

![image.png](https://i.postimg.cc/sx2WK6v3/image.png)


**`orders`**

![image.png](https://i.postimg.cc/T2H5fcKW/image.png)

b. Given the `products` table shown below:

[![image.png](https://i.postimg.cc/W18dpLXC/image.png)](https://postimg.cc/DSWv5DVP)

write a query that returns the price of each product at every store. Your result should look like this:

[![image.png](https://i.postimg.cc/s235K4Nd/image.png)](https://postimg.cc/FYnfH3vD)

In [3]:
# solution
#a
pd.read_sql_query("""SELECT name FROM customers
                        LEFT JOIN orders ON customers.id=orders.id
                        WHERE order_id IS NULL""", con)

,name
0,Afam
1,Deborah


In [4]:
#b
pd.read_sql_query("""SELECT id, 
                        MAX(CASE 
                            WHEN store = "store a" THEN price 
                            ELSE null 
                            END) AS "store a",
                        MAX(CASE 
                            WHEN store = "store b" THEN price 
                            ELSE null 
                            END) AS "store b",
                        MAX(CASE 
                            WHEN store = "store c" THEN price 
                            ELSE null 
                            END) AS "store c"
                        FROM products
                        GROUP BY id;
                        """,con)

,id,store a,store b,store c
0,0,10,11,12
1,1,33,35,34


## Qs 2: 

Given the tables `employee` and `department` shown below:

![image.png](https://i.postimg.cc/bw0J574p/image.png)

a. Write a query that generates the table shown below:

![image](https://i.postimg.cc/hjc3rdTK/image.pnghttps://i.postimg.cc/hjc3rdTK/image.png)

b. Write a query that determines the **highest earner** in **each department**.

c. Write a query that shows all the employees in each department **except** the **lowest earner**. i.e generate the table below

![image.png](https://i.postimg.cc/MHcmw7Q0/image.png)

In [5]:
# solution
#a
pd.read_sql_query("""SELECT name, salary, dept_name
                    FROM employee
                    LEFT JOIN department ON employee.dept_id=department.id
                    ORDER BY dept_name""",
                  con)

,name,salary,dept_name
0,Felix,33000,IT
1,Okafor,25000,IT
2,Gafar,41000,IT
3,Amanda,53000,IT
4,Steve,32000,Sales
5,Bola,31000,Sales
6,Babalola,30200,Sales
7,Adigun,30000,Sales


In [6]:
# solution here
#b
pd.read_sql_query("""SELECT name, Max(salary), dept_name
                        FROM employee
                        LEFT JOIN department ON employee.dept_id=department.id
                        GROUP BY dept_name""",
                  con)

,name,Max(salary),dept_name
0,Amanda,53000,IT
1,Steve,32000,Sales


In [7]:
# solution here
#c
pd.read_sql_query("""SELECT name, salary, dept_name
                        FROM employee
                        LEFT JOIN department ON employee.dept_id=department.id
                        WHERE salary != (SELECT MIN(salary) FROM employee)  
                        ORDER BY salary DESC""",
                  con)

,name,salary,dept_name
0,Amanda,53000,IT
1,Gafar,41000,IT
2,Felix,33000,IT
3,Steve,32000,Sales
4,Bola,31000,Sales
5,Babalola,30200,Sales
6,Adigun,30000,Sales


### Qs 3: Given the students table:

a. Write a query that creates a new column `percentage_score`.  The `percentage_score` is the mean of all the subjects for each student. Round the `percentage_score` to 2 decimal points and sort in descending order.

b. Write a query that selects **ONLY** the students that scored more than the mean of the `percentage_score`.

In [ ]:
# solution
#a
pd.read_sql_query("""SELECT matric_no, name, gender,
                        ROUND(((maths + english + physics)/3),2) AS percentage_score FROM students
                        ORDER BY percentage_score DESC"""
                  ,con)

,matric_no,name,gender,percentage_score
0,ux-012-013,Emeka,M,91.0
1,ux-012-020,Ahmed,M,88.0
2,ux-012-016,Esther,F,87.0
3,ux-012-014,Agnes,F,79.0
4,ux-012-018,Ogechi,F,75.0
5,ux-012-023,Dave,M,72.0
6,ux-012-015,Alimat,F,71.0
7,ux-012-019,Steve,M,67.0
8,ux-012-024,Seyi,M,66.0
9,ux-012-021,Samantha,F,56.0


In [9]:
# Write your solution here
#b
pd.read_sql_query("""SELECT matric_no, name, gender,
                        ROUND(((maths + english + physics)/3),2) percentage_score FROM students 
                        WHERE percentage_score > (SELECT AVG(ROUND(((maths + english + physics)/3),2)) percentage_score
                                                      FROM students)
                        """
                  ,con)

,matric_no,name,gender,percentage_score
0,ux-012-013,Emeka,M,91.0
1,ux-012-014,Agnes,F,79.0
2,ux-012-015,Alimat,F,71.0
3,ux-012-018,Ogechi,F,75.0
4,ux-012-016,Esther,F,87.0
5,ux-012-020,Ahmed,M,88.0
6,ux-012-023,Dave,M,72.0
